# Notebook Description

This notebook is meant to be a backtesting environment to analyze quantitative-based trading strategies in the US Stock Markets. This notebook integrates data via Microsoft Excel and then goes through a cleaning and transformation process. In addition, this notebook displays strategy results for the following metrics: "Final Amount", "Median Percent Returns", "Average Percent Returns", "Number of Trades", "Win Ratio", "Loss Ratio", "Average Gain per Trade", "Average Loss per Trade", and "Final Ratio". This notebook contains buy and sell code that creates indicators for any strategy being used, please see the disclaimer. As of right now, this notebook only contains one strategy and that is the 3 SMA Strategy. Any strategy can be easily substituted.


Last Updated: July 26, 2023

Data Source: Microsoft Excel

Date Range: January 1, 2017 to July 26, 2023

# Instructions For Use

- Step 1: Import Necessary Libraries
- Step 2: Select what theory you want to analyze on the portfolio. As of 7/27/23, there is only one theory (3 SMA)
- Step 3: Run the results code
- Step 4: Inside the "Select a Company.." block, change the 3 SMA values (halfway down the cell). The X, Y, Z values are currently set at 3, 15, and 18 days which has been optimized in the "Optimization of 3 SMA Strategy" 
- Step 5: When prompted, select one of the 10 companies in the portfolio. 
- Step 6: View your selected companies trades, performance, and results

# Import Necessary Libraries

In [4]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Strategy 1 - SMA - Buy/Sell Code

3 SMA Idealogy: 

Small MA > Middle MA > Large MA == BUY signal

Large MA > Middle MA > Small MA == SELL signal

In [5]:
def SMA(df, x, y, z):
    # Create the SMA values for the intervals of x days, y days, and z days
    df["small SMA"] = df["Close"].rolling(window=x).mean()
    df["int SMA"] = df["Close"].rolling(window=y).mean()
    df["large SMA"] = df["Close"].rolling(window=z).mean()

    # Trading Strategy CODE
    # 3 MA strategy (big, intermediate, and small)
    df["Signal"] = np.where((df["small SMA"] > df["int SMA"]) & (df["int SMA"] > df["large SMA"]), "Buy",
                            np.where((df["large SMA"] > df["int SMA"]) & (df["int SMA"] > df["small SMA"]), "Sell", "Hold"))

    # BUY/SELL SIGNAL CODE 
    # DISCLAIMER NOTE: This specific buy/sell signal code in this cell is not my own work. This code was generated through outside resources (ChatGPT) and it is not my work. The rest of the code in the Notebook and Cell IS indeed my own work.
    buy_date = None
    buy_price = None
    sell_date = None
    sell_price = None
    ignore_signals = False

    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        if ignore_signals:
            if row["Signal"] == "Buy":
                ignore_signals = False
        else:
            if row["Signal"] == "Buy" and buy_date is None:
                buy_date = index
                buy_price = row["Open"]
                ignore_signals = True
            elif row["Signal"] == "Sell" and buy_date is not None and sell_date is None:
                sell_date = df.index[df.index.get_loc(index) + 1]
                sell_price = df.loc[sell_date, "Open"]
                ignore_signals = True
                # Calculate the +/- amount and update the '+/- Amount' column
                plus_minus_amount = sell_price - buy_price
                df.loc[sell_date, "+/- Amount"] = plus_minus_amount
                
                 # Calculate the "Percent Return" and update the 'Percent Return' column
                percent_return = (plus_minus_amount / buy_price) * 100
                df.loc[sell_date, "Percent Return"] = percent_return
            
                # Reset the variables for the next pair of signals
                buy_date = None
                buy_price = None
                sell_date = None
                sell_price = None  
    return df

# Results Code

Overall recap of the trading strategy's performance for the chosen company.

Returns... 
- all trades
- all gains
- all loses
- outcome dataframe with final amounts, win/loss ratio, average gain/loss per trade, and final ratio

In [6]:
def results(df):
    # Breakdown of All trades
    results_df = df.dropna().copy()
    results_df.loc[:, "+/- Amount"] = results_df.loc[:, "+/- Amount"].round(2)
    
    # Breakdown of Gains
    gain_df = results_df[results_df["+/- Amount"] > 0]
    
    # Breakdown of Losses
    loss_df = results_df[results_df["+/- Amount"] < 0]

    # Analysis and Ratios of Tables
    ratio = {
        "Final Amount": results_df["+/- Amount"].sum().round(2),
        "Median Percent Returns": round(results_df["Percent Return"].median(),2),
        "Average Percent Returns": round(results_df["Percent Return"].mean(),2),
        "Number of Trades": len(results_df),
        "Win Ratio": [round(len(gain_df)/len(results_df), 2) * 100],
        "Loss Ratio": [round(len(loss_df)/len(results_df), 2) * 100],
        "Average Gain per Trade": [round(gain_df["+/- Amount"].mean(), 2)],
        "Average Loss per Trade": [round(loss_df["+/- Amount"].mean(), 2)],
        "Final Ratio": [abs(round((gain_df["+/- Amount"].sum() / len(gain_df)) / (loss_df["+/- Amount"].sum() / len(loss_df)), 2))]
    }
    ratio_df = pd.DataFrame(ratio, index=[df.name])

    return results_df, gain_df, loss_df, ratio_df

# Select a Specific Company & See the Results

- Data Cleaning for all 10 companies
- Incorporates SMA and Result functions into all 10 companies
- Only displays the results, gains, and losses data for the user-chosen company
- Displays the Outcome Dataframe full of ratio's and metrics on all 10 companies

In [7]:
# Company List
company = {"AVGO": "Broadcom", "CLX": "Clorox", "COST": "Costco", "CXW": "CoreCivic", "GPRO":"GoPro", 
           "HTHT": "H_World_Group", "NEOG": "Neogen", "PKG": "Packaging", "SAH": "Sonic", "TTC": "Toro"}

# Get the selected company from the user
print("Available companies include: AVGO, CLX, COST, CXW, GPRO, HTHT, NEOG, PKG, SAH, and TTC")
selected_company = input("Enter the company ticker symbol: ")

# Check if the selected company is valid
if selected_company in company.keys():
    
    # Dictionary to store the outcome data
    outcome_dict = {}

    for company_abbrv in company.keys():
        df = pd.read_excel(f"FIN-{company_abbrv}.xlsx")
        
        # Drop 0th row, make 1st row headers, and values in 2nd row
        df = pd.DataFrame(df.values[2:], columns=df.iloc[1])
        
        #Make All Columns Numeric
        df["Open"] = pd.to_numeric(df["High"], errors="coerce")
        df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
        df["Low"] = pd.to_numeric(df["Low"], errors="coerce")
        df["High"] = pd.to_numeric(df["High"], errors="coerce")
        
        # Change Date Column from imported to proper format and set as index
        df["Date"] = pd.to_numeric(df["Date"], errors="coerce")
        df["Date"] = pd.to_datetime(df["Date"], origin="1899-12-30", unit="D").dt.strftime("%m/%d/%y")
        df = df.set_index("Date")
        
        # Reorder columns
        desired = ["Open", "Close", "Low", "High", "Volume"]
        df = df.reindex(columns=desired)
        
        # Set the name of the dataframe
        df.name = company[company_abbrv] 
        
        #INCORPORATE THE BUY/SELL CODE & RESULTS CODE
        # Calculate SMA and perform trading strategy
        df = SMA(df, 3, 15, 18)
        # Calculate results for each company
        results_df, gain_df, loss_df, ratio_df = results(df)
        
        
        # ONLY display the trades, gains, losess dataframes for only the selected company
        # This way we aren't cluttering the output and making it hard to view the results
        if company_abbrv == selected_company:
            print("Company:", df.name)
            print("")
            print("The total account would be up: $", df["+/- Amount"].sum().round(2))
            print("")
            print("Analysis of Trading Strategy:")
            display(ratio_df)
            print("Breakdown of All Trades:")
            display(results_df)
            print("Breakdown of All Gains:")
            display(gain_df)
            print("Breakdown of All Losses:")
            display(loss_df)

        # Append results to the outcome dictionary
        outcome_dict[company_abbrv] = {
            "Company": df.name,
            "Final Amount": results_df["+/- Amount"].sum().round(2),
            "Median Percent Returns": round(results_df["Percent Return"].median(),2),
            "Average Percent Returns": round(results_df["Percent Return"].mean(),2),
            "Number of Trades": len(results_df),
            "Win Ratio": ratio_df.iloc[0]["Win Ratio"],
            "Loss Ratio": ratio_df.iloc[0]["Loss Ratio"],
            "Average Gain per Trade": ratio_df.iloc[0]["Average Gain per Trade"],
            "Average Loss per Trade": ratio_df.iloc[0]["Average Loss per Trade"],
            "Final Ratio": ratio_df.iloc[0]["Final Ratio"]
        }

    # Convert the outcome dictionary to a DataFrame
    outcome_df = pd.DataFrame.from_dict(outcome_dict, orient="index")

    # Display the outcome DataFrame for all companies
    print("Outcome DataFrame for All Companies:")
    display(outcome_df)
    
    # Calculate the total strategy performance for all ten companies
    print("The total portfolio value for this 3-SMA Strategy is: $", outcome_df["Final Amount"].sum().round(2))
    print("The median total portfolio returns for this 3-SMA Strategy is: ", round(outcome_df["Median Percent Returns"].median(),2),"%")
    print("The average total portfolio returns for this 3-SMA Strategy is: ", round(outcome_df["Average Percent Returns"].mean(),2),"%")

else:
    print("Invalid company ticker symbol. Please try again.")

Available companies include: AVGO, CLX, COST, CXW, GPRO, HTHT, NEOG, PKG, SAH, and TTC
Enter the company ticker symbol: AVGO
Company: Broadcom

The total account would be up: $ 445.25

Analysis of Trading Strategy:


,Final Amount,Median Percent Returns,Average Percent Returns,Number of Trades,Win Ratio,Loss Ratio,Average Gain per Trade,Average Loss per Trade,Final Ratio
Broadcom,445.26,0.57,3.15,34,53.0,47.0,37.52,-14.38,2.61


Breakdown of All Trades:


1,Open,Close,Low,High,Volume,small SMA,int SMA,large SMA,Signal,+/- Amount,Percent Return
Date,,,,,,,,,,,
04/03/17,219.9900,218.80,216.6623,219.9900,1921017,219.266667,220.229333,221.013889,Sell,14.20,6.900238
06/27/17,241.9900,235.33,235.2300,241.9900,3358160,241.113333,243.106667,244.844444,Sell,15.80,6.985278
08/11/17,245.2400,244.17,238.7000,245.2400,1697612,244.683333,250.092000,250.684444,Sell,2.64,1.088211
09/08/17,246.3970,244.11,243.1001,246.3970,1810040,246.680000,249.108667,249.482778,Sell,-12.51,-4.832954
12/13/17,263.2900,262.03,260.1800,263.2900,2694628,260.226667,268.960000,269.873889,Sell,12.45,4.963323
01/16/18,267.0000,263.25,261.6600,267.0000,3352819,263.680000,264.674000,264.498889,Hold,-4.76,-1.751545
03/21/18,250.5900,246.00,240.0600,250.5900,5491222,244.436667,252.032667,251.761667,Hold,-18.85,-6.995992
06/28/18,247.3200,244.90,242.2650,247.3200,3324128,247.850000,258.966667,259.553333,Sell,2.88,1.178203
10/15/18,235.3300,232.47,230.0400,235.3300,3009936,231.440000,242.527333,243.355000,Sell,14.10,6.373457


Breakdown of All Gains:


...

Breakdown of All Losses:


1,Open,Close,Low,High,Volume,small SMA,int SMA,large SMA,Signal,+/- Amount,Percent Return
Date,,,,,,,,,,,
09/08/17,246.3970,244.11,243.1001,246.3970,1810040,246.680000,249.108667,249.482778,Sell,-12.51,-4.832954
01/16/18,267.0000,263.25,261.6600,267.0000,3352819,263.680000,264.674000,264.498889,Hold,-4.76,-1.751545
03/21/18,250.5900,246.00,240.0600,250.5900,5491222,244.436667,252.032667,251.761667,Hold,-18.85,-6.995992
08/12/19,278.2100,275.64,272.5400,278.2100,2003969,274.116667,285.834667,286.858333,Sell,-4.81,-1.699527
09/30/19,276.7200,276.07,272.5000,276.7200,2020410,275.706667,286.276000,287.204444,Sell,-9.18,-3.210913
01/07/20,316.5765,312.64,312.2500,316.5765,1863062,313.516667,318.962000,319.298333,Sell,-4.77,-1.485452
02/20/20,315.6900,309.24,306.1200,315.6900,2552088,311.893333,315.064667,315.045556,Hold,-4.19,-1.309866
07/27/20,312.9600,312.68,307.4850,312.9600,2053608,309.380000,314.084000,314.347222,Sell,-8.03,-2.501636
11/02/20,355.3900,351.25,348.3450,355.3900,1170829,351.963333,367.800667,369.415000,Sell,-15.61,-4.207547


Outcome DataFrame for All Companies:


,Company,Final Amount,Median Percent Returns,Average Percent Returns,Number of Trades,Win Ratio,Loss Ratio,Average Gain per Trade,Average Loss per Trade,Final Ratio
AVGO,Broadcom,445.26,0.57,3.15,34,53.0,47.0,37.52,-14.38,2.61
CLX,Clorox,87.82,1.47,2.26,29,66.0,34.0,8.46,-7.29,1.16
COST,Costco,338.71,0.53,3.45,31,61.0,39.0,22.15,-6.85,3.23
CXW,CoreCivic,-11.65,-3.57,-2.10,31,29.0,71.0,1.55,-1.17,1.33
GPRO,GoPro,-9.79,-4.19,-1.96,34,32.0,68.0,0.78,-0.80,0.97
HTHT,H_World_Group,0.57,-2.67,1.89,33,48.0,52.0,4.33,-4.04,1.07
NEOG,Neogen,-16.94,-3.41,-1.13,34,29.0,71.0,3.69,-2.25,1.65
PKG,Packaging,25.98,-0.85,0.97,36,44.0,56.0,7.50,-4.70,1.60
SAH,Sonic,13.27,-3.83,3.44,33,42.0,58.0,5.00,-2.99,1.67
TTC,Toro,12.08,-0.62,0.67,34,47.0,53.0,5.24,-3.98,1.31


The total portfolio value for this 3-SMA Strategy is: $ 885.31
The median total portfolio returns for this 3-SMA Strategy is:  -1.76 %
The average total portfolio returns for this 3-SMA Strategy is:  1.06 %


# Optimization Code Results

Optimization Code = view "Optimize of 3 SMA Strategy" in Jupyter

The 3 values for the SMA code have been optimized looking at the same time frame from Janaury 1, 2017 to July 26, 2023 for Apple.

It was found that for the ranges of small(0-6), medium (6-16), and large SMA (16,30) ... the most optimal combination was 3, 15, and 18.

Apple was chosen because it was a company outside of our portfolio to ensure we didn't overtrain our trading model to this specific data. The timeframe was held constant as we wanted to have a constant through the fluctuating and market conditions pre, during, and post COVID-19.